# Slide Set 9 - Cylinder flow

This script plots streamlines and potential lines for the flow past a cylinder. It results from the superposition of a dipole and a parallel flow.

## Preparation

To prepare the plot, several helper functions have to be defined. This is done in the following cell.
First, the two libraries numpy and matplotlib.pyplot have to be imported in order to user their functionality. Note, these libraries are helpful in many scenarios, when data should be plotted. Next, helper classes that provide the complex velocity for the source-/sink-flow and the parallel flow are implemented. Finally, a function providing the meshgrid for the calculations is implemented.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define important colors
TUMOrange = "#E37222"
TUMBlue = "#3070b3"
TUMGreen = "#A2AD00"

# Define helper classes to get the complex velocity of elementary flows
class ParallelFlow:

    def __init__(self, velocity_x: float, velocity_y: float) -> None:
        """Initializer for the parallel flow class. Stores relevant quantities as class members.

        Args:
            velocity_x (float): Cartesion component of the velocity pointing in x-direction.
            velocity_y (float): Cartesion component of the velocity pointing in y-direction.
        """
        self.velocity_x = velocity_x
        self.velocity_y = velocity_y

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Return the complex velocity of the parallel flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        return self.velocity_x - 1.0j * self.velocity_y


class DipoleFlow:

    def __init__(self, dipole_moment, translation_x = 0.0, translation_y = 0.0) -> None:
        """Initializer for the dipole class. Stores relevant quantities as class members.

        Args:
            dipole_moment (float): The dipole moment.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.dipole_moment = dipole_moment
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.factor = self.dipole_moment / np.pi

    def get_complex_velocity(self, z):
        """Returns the complex velocity of the source-/sink-flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z_translated = z - self.translation_x - 1.0j * self.translation_y
        return - self.factor / (z_translated*z_translated)
    
def get_grid_quantities(x_min: float, x_max: float, y_min: float, y_max: float, resolution_x: int, resolution_y: int):
    """Returns the meshgrid used for the calculations.

    Args:
        x_min (float): The minimum x value.
        x_max (float): The maximum x value.
        y_min (float): The minimum y value.
        y_max (float): The maximum y value.
        resolution_x (int): The number of points to discretize in x direction.
        resolution_y (int): The number of points to discretize in y direction.

    Returns:
        List[np.array]: The numpy array containing the meshgrid.
    """
    x_linspace = np.linspace(x_min, x_max, resolution_x)
    y_linspace = np.linspace(y_min, y_max, resolution_y)
    X_meshgrid, Y_meshgrid = np.meshgrid(x_linspace, y_linspace)
    Z_meshgrid = X_meshgrid + 1.0j * Y_meshgrid
    return X_meshgrid, Y_meshgrid, Z_meshgrid

class PotentialVortexFlow:

    def __init__(self, gamma: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the vortex flow class. Stores relevant quantities as class members.

        Args:
            gamma (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.gamma = gamma
        self.factor = self.gamma / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the vortex flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z = z - self.translation_x - 1.0j * self.translation_y
        return - 1.0j * self.factor / z

## Definition of domain

Here, relevant information to describe the domain is provided. It is used to generate the meshgrid for the calculations.

In [ ]:
x_min = -2.0
x_max = 2.0
y_min = -2.0
y_max = 2.0
resolution_per_unit_length = 20
resolution_x = ( x_max - x_min ) * resolution_per_unit_length
resolution_y = ( y_max - y_min ) * resolution_per_unit_length

X_meshgrid, Y_meshgrid, Z_meshgrid = get_grid_quantities(x_min, x_max, y_min, y_max, int(resolution_x), int(resolution_y))

## Calculation of the complex velocity

Initializes the relevant source, sink and parallel flow objects to calculate the complex velocity.

In [ ]:
u_infty = 1.0
radius = 0.5
angle = 0.0 * np.pi

dipole_moment = u_infty * np.pi * radius**2

gamma = np.imag(np.exp(1.0j * angle))
circulation = 4.0 * np.pi * u_infty * radius * gamma

dipole = DipoleFlow(dipole_moment)
inflow = ParallelFlow(u_infty, 0.0)
potential_vortex = PotentialVortexFlow(circulation)

complex_velocity = inflow.get_complex_velocity(Z_meshgrid) + dipole.get_complex_velocity(Z_meshgrid) + potential_vortex.get_complex_velocity(Z_meshgrid)
u = np.real(complex_velocity)
v = - np.imag(complex_velocity)

## Plotting of the results

Plot the streamlines and potential lines.
Streamlines are plotted in TUMBlue.
Potential lines are plotted in TUMOrange.
The stagnation streamline is plotted in red.

In [ ]:
fig, ax = plt.subplots()

x_linspace = np.linspace(x_min, x_max, 15)
y_linspace = np.linspace(x_min, x_max, 20)

# Plot the streamlines.

# The streamline seeds describe all the points where streamlines start. Thus, only streamline the go through these points are drawn.
# This may heavily influence the look of the plot!
# In this case only points to the left (*xmin refers to the left, replace with e.g. 0.0 to also obtain streamlines inside the oval)
# of the oval are choosen. Thus, no streamlines inside the oval show.
# In this case, they are equally distributed along the y direction.
streamline_seeds = np.stack([np.ones_like(y_linspace)*x_min, y_linspace], axis=-1)

ax.streamplot(
    X_meshgrid, Y_meshgrid, u, v,
    density=[1.0, 1.0],
    minlength=1.0, maxlength=20.0,
    broken_streamlines=False,
    start_points=streamline_seeds,
    color=TUMBlue,
)

# Plot the potential lines.

# The potential line seed describe all points where potential lines start. Here, similar remarks as above apply.
potential_line_seeds_1 = np.stack([x_linspace, np.ones_like(x_linspace)*y_min], axis=-1)
potential_line_seeds_2 = np.stack([x_linspace, np.ones_like(x_linspace)*y_max], axis=-1)
potential_line_seeds = np.concatenate([potential_line_seeds_1, potential_line_seeds_2], axis=0)
ax.streamplot(
    X_meshgrid, Y_meshgrid, -v, u,
    density=[0.2, 0.2],
    minlength=1.0, maxlength=20.0,
    broken_streamlines=False,
    start_points=potential_line_seeds,
    arrowstyle = "-",
    color=TUMOrange,
)

# Plot circle
# ax.plot(
#     [x_min, x_max], [0.0, 0.0],
#     color="red",
#     linewidth=3.0,
# )
phi_linspace = np.linspace(0.0, 2.0*np.pi, 100)
ax.plot(
    radius * np.cos(phi_linspace), radius * np.sin(phi_linspace),
    color="red",
    linewidth=3.0,
)
phi_array = np.array([angle, np.pi-angle])
ax.scatter(
    radius * np.cos(phi_array), radius * np.sin(phi_array),
    color=TUMGreen,
    linewidth=3.0,
)

ax.set_aspect('equal', 'box')
plt.show()

Plotting streamlines and potential lines without prescribing any seed, and by that not imposing prior knowledge, is also possible

In [ ]:
plt.close()

fig, ax = plt.subplots()

# Plot the streamlines.
ax.streamplot(
    X_meshgrid, Y_meshgrid, u, v,
    density=[1.0, 1.0],
    color=TUMBlue,
)

# Plot the potential lines.
ax.streamplot(
    X_meshgrid, Y_meshgrid, -v, u,
    density=[0.2, 0.2],
    arrowstyle = "-",
    color=TUMOrange,
)

ax.set_aspect('equal', 'box')
plt.show()

### Pressure distribution along the cylinder's surface.

Here, we calculate and plot the pressure coefficient along the surface of the cylinder.

In [ ]:
phi_linspace = np.linspace(0.0, np.pi, 1000)
x_circle = radius * np.cos(phi_linspace)
y_circle = radius * np.sin(phi_linspace)
z_circle = x_circle + 1.0j * y_circle
complex_velocity_surface = inflow.get_complex_velocity(z_circle) + dipole.get_complex_velocity(z_circle) + potential_vortex.get_complex_velocity(z_circle)
q_squared_surface = np.absolute(complex_velocity_surface)**2
pressure_coefficient_surface_upper = 1.0 - q_squared_surface / u_infty**2

phi_linspace = np.linspace(np.pi, 2.0 * np.pi, 1000)
x_circle = radius * np.cos(phi_linspace)
y_circle = radius * np.sin(phi_linspace)
z_circle = x_circle + 1.0j * y_circle
complex_velocity_surface = inflow.get_complex_velocity(z_circle) + dipole.get_complex_velocity(z_circle) + potential_vortex.get_complex_velocity(z_circle)
q_squared_surface = np.absolute(complex_velocity_surface)**2
pressure_coefficient_surface_lower = 1.0 - q_squared_surface / u_infty**2

plt.close()
fig, ax = plt.subplots()
# ax.set_aspect('equal', 'box')

ax.scatter(np.real(z_circle), -pressure_coefficient_surface_upper, c=TUMGreen)
ax.scatter(np.real(z_circle), -pressure_coefficient_surface_lower, c=TUMOrange)

x_s1 = np.real(radius * np.exp(1.0j * angle))
x_s2 = np.real(radius * np.exp(1.0j * (np.pi - angle)))

ax.scatter([x_s1, x_s2], [-1.0, -1.0], c=TUMBlue, marker="*", s=200)

ax.legend(["Upper half of the cylinder", "Lower half of the cylinder", "Stagnation points"])
ax.set_xlabel("x coordinate of point on the cylinder")
ax.set_ylabel("- C_p value")
ax.set_ylim(ymin=-1.2)

plt.show()